# Estudo dos tamanhos das *bounding boxes* VDAO

Este notebook tem como objetivo fazer um levantamento das áreas dos bounding boxes da base [**Video Database of Abandoned Objects in a Cluttered Industrial Environment (VDAO)**](http://www02.smt.ufrj.br/~tvdigital/database/objects/page_01.html).

Parte do código do projeto [VDAO_Access](https://github.com/rafaelpadilla/DeepLearning-VDAO/tree/master/VDAO_Access) foi utilizado como suporte para este notebook.

As áreas foram obtidas através dos arquivos de anotação distribuídos juntos com a VDAO.

In [1]:
%load_ext autoreload

In [2]:
# Importando os pacotes necessários
import sys
import os
import numpy as np
import glob
import re
import pandas as pd
# Import module utils
sys.path.append(os.path.join(os.getcwd(),'..'))
from utils import _init_paths
from utils.generic_utils import get_files_paths
from utils.definitions import *
# Import classes Annotation and VDAOVideo
sys.path.append(os.path.join(os.getcwd(),'..','..'))
from VDAO_Access.Annotation import Annotation
from VDAO_Access.VDAOVideo import VDAOVideo

## Fazendo a leitura dos arquivos de anotação

Para a execução deste notebook, é necessário fazer o downlod dos arquivos de anotação, indetificando o caminho da pasta na variável `vdao_dir`

In [3]:
min_bounding_box_area_to_consider = 0

In [4]:
# Definindo pasta com os vídeos e arquivos de anotação da VDAO
vdao_dir = vdao_videos_dir['train']
# Obtém todos os arquivos de anotação
annot_files = get_files_paths(vdao_dir, 'txt')

all_areas = {}
# Para cada arquivo, obtemos um objeto Annotation, que facilita a obtenção da área dos bounding boxes
for file in annot_files:
    # Obtém objeto Annotation e as respectivas áreas contidas no arquivo de anotação
    annot = Annotation(annotationFilePath=file)
    areas_and_proportions = annot.get_object_areas_and_proportions()
    # Adiciona no dicionário all_areas cada uma das áreas
    for k, v in areas_and_proportions.items():
        v = [i for i in v if i[0] >= min_bounding_box_area_to_consider]
        if k not in all_areas:
            all_areas[k] = []
        all_areas[k] = all_areas[k] + v

# Adicionando todas os bounding boxes de todas as classes
all_sizes = []
for _, v in all_areas.items():
    all_sizes += [s for s in v]
all_areas['(Todos bounding boxes)'] = all_sizes

# Para cada classe, obtém métricas (área mínima, área máxima, área média, etc)
stats_by_classes = []
for k, v in all_areas.items():
    # Obtém todas as áreas
    areas = [i[0] for i in v]
    # Obtém todas proporções desconsiderando aquelas (height == 0)
    props = [i[1] for i in v if i[1] != None]
    stats_by_classes.append({
        'object': k,
        'area_min': np.min(areas),
        'area_max': np.max(areas),
        'area_avg': np.average(areas),
        'prop_min': np.min(props),
        'prop_max': np.max(props),
        'prop_avg': np.average(props),
        'total_bb': len(areas)
    })

## Exibindo áreas dos *bounding boxes* para cada classe de objeto

In [5]:
# Criamos um data frame para exibição dos resultados
df = pd.DataFrame(stats_by_classes) 
df.rename(columns={'object':'objeto',
                   'area_min':'área mínima',
                   'area_max':'área máxima',
                   'area_avg':'área média',
                   'prop_min': 'prop w/h mínima',
                   'prop_max':'prop w/h máxima',
                   'prop_avg':'prop w/h média',
                   'total_bb':'bounding boxes'},inplace=True)
df.set_index('objeto', inplace=True)
# Mostrando a maior e menor bounding box
print('Maior bounding box: %d' % df['área máxima'].max())
print('Menor bounding box: %d' % df['área mínima'].min())
# Mostrando a maior e menor proporção
print('Maior proporção (width/height): %d' % df['prop w/h máxima'].max())
print('Menor proporção (width/height): %d' % df['prop w/h mínima'].min())
df

Maior bounding box: 697680
Menor bounding box: 0
Maior proporção (width/height): 24
Menor proporção (width/height): 0


,área média,área máxima,área mínima,prop w/h média,prop w/h máxima,prop w/h mínima,bounding boxes
objeto,,,,,,,
shoe,37980.598077,107802,0,0.862224,2.425641,0.059172,12169
darkBlueBox,55049.803807,315200,0,1.330022,8.700000,0.029412,18966
cameraBox,62355.288587,378225,0,1.266958,24.000000,0.031496,28144
whiteJar,16168.074445,32224,0,0.536846,0.927273,0.046512,10988
brownBox,41322.761228,145280,0,1.059456,1.689531,0.125000,11534
caixa,28169.185674,43432,0,1.104948,1.587838,0.113924,2806
pinkBottle,14294.319691,38824,0,1.116071,6.111111,0.013245,17592
toalha,92129.322861,461892,0,0.726982,1.916667,0.038710,32085
blackCoat,106365.089666,594940,0,1.337207,3.062201,0.051282,31517


In [6]:
# Define a dictionary gathering only the areas of each class
all_areas_only = {}
for k, v in all_areas.items():
    all_areas_only[k] = [i[0] for i in v]
# Define a dictionary gathering only the sizes 
all_sizes_only = []
for s in all_sizes:
    all_sizes_only.append(s[0])   

## Histogramas

In [7]:
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

### Separados por classes:

In [8]:
%matplotlib inline
# Importing matplotlib and generate individual histogram for each class
import matplotlib.pyplot as plt

   
def plot_histogram(object_class):
    n, bins, patches = plt.hist(x=all_areas_only[object_class],bins='auto', color='#0504aa',alpha=0.7, rwidth=0.85)
    plt.xlabel('sizes of bounding boxes')
    plt.ylabel('occurrences')
    plt.title('Histogram %s' % object_class)
    plt.grid(True)
    plt.show()

In [9]:
# Apresentando em um widget para facilitar a visualização
from ipywidgets import interactive

interactive_plot = interactive(plot_histogram, object_class=sorted(all_areas.keys()))
interactive_plot

interactive(children=(Dropdown(description='object_class', options=('(Todos bounding boxes)', 'CaixaMa', 'back…

## Apresentação em forma de tabela

In [10]:
# Obtendo proporções por área
proportions_by_areas = {}
for s in all_sizes:
    if s[0] not in proportions_by_areas.keys():
        proportions_by_areas[s[0]] = []
    proportions_by_areas[s[0]].append(s[1])
    
metrics_by_areas = {}
for k, v in proportions_by_areas.items():
    # Para os casos de área = 0, proporção é None
    if None in v:
        metrics_by_areas[k] = {
            'prop_min': np.nan,
            'prop_max': np.nan,
            'prop_avg': np.nan,
            'total_occurencies': len(v)
        }
    else:
        metrics_by_areas[k] = {
            'prop_min': np.min(v),
            'prop_max': np.max(v),
            'prop_avg': np.average(v),
            'total_occurencies': len(v)
        }

A seguir os tamanhos dos **bounding boxes** são exibidos em forma de tabela. Nota-se a existência de 1611 **bounding boxes** com área igual a 0.

In [11]:
import collections
occurrences = collections.Counter(all_sizes_only)
dict_occurrences = collections.OrderedDict(sorted(occurrences.items()))
for k, v in dict_occurrences.items():
    dict_occurrences[k] = [v, metrics_by_areas[k]['prop_min'], metrics_by_areas[k]['prop_max'], metrics_by_areas[k]['prop_avg']]
df = pd.DataFrame.from_dict(dict_occurrences, orient='index')
df.rename(columns={0:'ocorrências'},inplace=True)
df.rename(columns={1:'prop mín'},inplace=True)
df.rename(columns={2:'prop max'},inplace=True)
df.rename(columns={3:'prop média'},inplace=True)
# Adiciona coluna com porcentagem de ocorrências
df['% do total'] = ((df.ocorrências / df.ocorrências.sum())*100).round(4).astype(str) + '%'
df.index.names = ['área bounding box']
df.to_csv('../all_areas.csv')
df

,ocorrências,prop mín,prop max,prop média,% do total
área bounding box,,,,,
0,1611,NaN,NaN,NaN,0.2922%
4,1,4.000000,4.000000,4.000000,0.0002%
8,1,0.500000,0.500000,0.500000,0.0002%
19,8,0.052632,0.052632,0.052632,0.0015%
20,1,1.250000,1.250000,1.250000,0.0002%
22,1,0.181818,0.181818,0.181818,0.0002%
23,2,0.043478,0.043478,0.043478,0.0004%
24,7,0.375000,24.000000,3.791667,0.0013%
25,2,1.000000,1.000000,1.000000,0.0004%


## Arquivos com **bounding boxes** vazios

Alguns arquivos de anotação possuem bounding boxes vazios. Abaixo segue uma lista com os nomes dos arquivos que possuem bounding boxes com área 0.

In [12]:
# Percorrendo (novamente) todos arquivos de anotação para encontrar aqueles com bounding boxes vazios
empty_bb_files = {}
for file in annot_files:
    all_areas = {}
    annot = Annotation(annotationFilePath=file)
    areas = annot.GetObjectsArea()
    empty_bb = 0
    for k, v in areas.items():
        if k not in all_areas:
            all_areas[k] = []
        all_areas[k] = all_areas[k] + v
        empty_bb += v.count(0)
    if empty_bb > 0:
        empty_bb_files[os.path.basename(file)] = empty_bb
        
df = pd.DataFrame.from_dict(empty_bb_files, orient='index')
df.rename(columns={0:'bounding boxes vazios'},inplace=True)
df.index.names = ['área bounding box']
df

,bounding boxes vazios
área bounding box,
obj-sing-amb-part01-video01.txt,1
obj-sing-amb-part01-video02.txt,2
obj-sing-amb-part01-video04.txt,145
obj-sing-amb-part01-video05.txt,30
obj-sing-amb-part01-video06.txt,20
obj-sing-amb-part01-video07.txt,20
obj-sing-amb-part01-video08.txt,15
obj-sing-amb-part01-video09.txt,20
obj-sing-amb-part01-video10.txt,5
